# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
Orders = pd.read_csv('../Datasets_as_CSV/Orders.csv')
Orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
Orders_aggexp = Orders.groupby(['CustomerID','Country'],as_index=False)['amount_spent'].sum()
Orders_aggexp

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [5]:
q_95 = Orders_aggexp['amount_spent'].quantile(0.95)
q_75 = Orders_aggexp['amount_spent'].quantile(0.75)

In [6]:
def customers_classifier(num):
    if num >= q_95:
        return 'VIP_customer'
    elif (num < q_95) & (num > q_75):
        return 'Pref_customer'
    else:
        return 'Std_customer'
    
Orders_aggexp['Custom_Label'] = Orders_aggexp['amount_spent'].apply(customers_classifier)

Orders_aggexp

,CustomerID,Country,amount_spent,Custom_Label
0,12346,United Kingdom,77183.60,VIP_customer
1,12347,Iceland,4310.00,Pref_customer
2,12348,Finland,1797.24,Pref_customer
3,12349,Italy,1757.55,Pref_customer
4,12350,Norway,334.40,Std_customer
...,...,...,...,...
4342,18280,United Kingdom,180.60,Std_customer
4343,18281,United Kingdom,80.82,Std_customer
4344,18282,United Kingdom,178.05,Std_customer
4345,18283,United Kingdom,2094.88,Pref_customer


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
(Orders_aggexp[Orders_aggexp['Custom_Label']=='VIP_customer']).groupby('Country').count().sort_values(by='Custom_Label',ascending=False)

,CustomerID,amount_spent,Custom_Label
Country,,,
United Kingdom,178,178,178
Germany,11,11,11
France,9,9,9
Switzerland,3,3,3
Spain,2,2,2
Portugal,2,2,2
Japan,2,2,2
EIRE,2,2,2
Finland,1,1,1


In [41]:
Orders_aggexp.groupby(['Custom_Label','Country']).size()['VIP_customer'].sort_values(ascending=False)

Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
EIRE                 2
Japan                2
Portugal             2
Spain                2
Norway               1
Netherlands          1
Singapore            1
Finland              1
Sweden               1
Denmark              1
Cyprus               1
Channel Islands      1
Australia            1
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [63]:
Orders_aggexp.groupby('Custom_Label')['Country'].describe()

,count,unique,top,freq
Custom_Label,,,,
Pref_customer,869,25,United Kingdom,756
Std_customer,3260,33,United Kingdom,2987
VIP_customer,218,17,United Kingdom,178
